In [1]:
from platform import python_version
print(python_version())

3.8.8


In [2]:
import sys
print(sys.version)

3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]


In [3]:
import tensorflow as tf
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17856244228974374145
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7055540224
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12425706403925108314
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 980M, pci bus id: 0000:01:00.0, compute capability: 5.2"
xla_global_id: 416903419
]


In [6]:
import pandas as pd

In [7]:
df = pd.read_json("mappingv3.json")

In [8]:
df.shape

(180, 3)

In [9]:
df.head()

,action_instances,id,nl_command_statment
0,"[{'options': [{'id': 601225, 'name': 'new emai...",101,Pay the invoice that I will receive from conta...
1,"[{'options': [{'id': 601225, 'name': 'new emai...",102,Pay in the bank my purchases that I will soon ...
2,"[{'options': [{'id': 601225, 'name': 'new emai...",105,Make a payment using my bank account when I re...
3,"[{'options': [{'id': 601225, 'name': 'new emai...",106,Pay my Good Sports Store account balance using...
4,"[{'options': [{'id': 601225, 'name': 'new emai...",107,Wait for a new email from the sender contact@g...


In [ ]:
str(df["action_instances"][0]).split("name")

In [ ]:
df["action_instances"][0]

In [ ]:
len(df["action_instances"][0])

In [ ]:
type(df["action_instances"][0][0])

In [ ]:
df["action_instances"][0][0]

In [ ]:
str(df["action_instances"][0])

In [ ]:
df["action_instances"][0]

In [ ]:
str(df["action_instances"][0]).split("\'name\': \'")

In [ ]:
data = str(df["action_instances"][0]).split("\'name\': \'")[1:]
data

In [ ]:
for everyData in data:
    print(everyData.find("'"))
    val = everyData.find("'")
    print(everyData[:val])

In [ ]:
actions_val = []
for everyCommand in df["action_instances"]:
    data = str(everyCommand).split("\'name\': \'")[1:]
    temp = []
    for everyData in data:
        val = everyData.find("'")
        temp.append(everyData[:val])
    actions_val.append(temp)    

In [ ]:
actions_val

In [ ]:
df["action_instances"][10]

In [ ]:
df["nl_command_statment"].iloc[10]

In [ ]:
import itertools
s = ",".join(itertools.chain(*actions_val))

In [ ]:
len(set(s.split(",")))

In [ ]:
for val in actions_val:
    print(len(val))

In [ ]:
temp = []
for val in actions_val:
    temp.append(len(val))

In [ ]:
max(temp), min(temp)

In [ ]:
temp.index(14)

In [ ]:
actions_val[95]

In [ ]:
df["action_instances"][95]

In [ ]:
df["nl_command_statment"].iloc[95]

In [ ]:
df["only_actions"] = actions_val

In [ ]:
df.head()

In [ ]:
# df["new_input"] = "<boF> " + df["nl_command_statment"] 
df["new_input"] = df["nl_command_statment"] 

In [ ]:
df.head()

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

input_tokenizer = Tokenizer()
input_tokenizer.fit_on_texts(df['new_input'])
Source_vocabulary = input_tokenizer.word_index.keys()

In [ ]:
output_tokenizer = Tokenizer()
output_tokenizer.fit_on_texts( df['only_actions'] )
target_vocabulary = output_tokenizer.word_index.keys()

In [ ]:
input_int = input_tokenizer.texts_to_sequences(df['new_input'])
output_int = output_tokenizer.texts_to_sequences(df['only_actions'])

In [ ]:
max_source_seq_length =  len(max( input_int ,key=len))

max_target_seq_length = len(max( output_int ,key=len))

num_source_tokens = len(input_tokenizer.word_counts)

num_target_tokens = len(output_tokenizer.word_counts)

number_of_samples = df.shape[0]


print('Number of samples:', number_of_samples )
print('Number of unique source language tokens:', num_source_tokens)
print('Number of unique target language tokens:', num_target_tokens)
print('Max sequence length of source language:', max_source_seq_length)
print('Max sequence length of target language:', max_target_seq_length)
print("Source Vocabulary",list(Source_vocabulary)[:10])
print("Target Vocabulary",list(target_vocabulary)[:10])

In [ ]:
import fasttext
import fasttext.util

# English Embeddings 
fasttext_model = fasttext.load_model(r"D:\jupyter_notebook_project\NUIG\SEM 2\advance NLP\assignmentsssssssss\assignment 3\cc.en.300.bin")
# fasttext_dict = {word: model.get_word_vector(word) for word in model.get_words()}
# del model

In [ ]:
import numpy as np

pad_input = np.zeros((max_source_seq_length, num_source_tokens), dtype='float32')

pad_output = np.zeros((max_target_seq_length, num_target_tokens), dtype='float32')

In [ ]:
import tensorflow as tf

In [ ]:
from keras.utils.vis_utils import plot_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import losses
from tensorflow.keras.layers import Input, LSTM, Dense,Flatten, TimeDistributed, SimpleRNN, Flatten , Reshape, Embedding
from tensorflow.keras.models import Model, Sequential

embed_mat_english = np.zeros((num_source_tokens+1, 300))
for value, position in input_tokenizer.word_index.items():
    embedding_vect = fasttext_model.get_word_vector(value)
    if embedding_vect is not None:
        embed_mat_english[position] = embedding_vect
    else:
        print("Hits value " , value ," position " , position)

english_layer_embedding = Embedding(num_source_tokens+1, 300, 
                         weights = [embed_mat_english],
                            input_length = max_source_seq_length , 
                                    trainable = False,
                                mask_zero = True) #using pre-trained embeddings

In [ ]:
max_target_seq_length , num_target_tokens

In [ ]:
#pad the sequences to the same length
pad_input_val = pad_sequences(input_int, maxlen=max_source_seq_length,padding='post')
print("endcoder input shape", pad_input_val.shape)

#add padding for encoder after the sentence integers
pad_output_val = pad_sequences(output_int, maxlen=num_target_tokens,padding='post' )
print("decoder input shape", pad_output_val.shape)

In [ ]:
reshaped_input = pad_input_val.reshape(180, 56, 1)
reshaped_output = pad_output_val.reshape( 180, 73 )

In [ ]:
reshaped_input.shape, reshaped_output.shape

In [ ]:
reshaped_output

In [ ]:
reshaped_output[:1 , ]

In [ ]:
reshaped_output[:1 , ].shape

In [ ]:
reshaped_output = to_categorical(reshaped_output[:1 , ])

In [ ]:
reshaped_output.shape

In [ ]:
reshaped_output[:1 , ]

In [ ]:
tdd = Input(shape= (56) )
x = english_layer_embedding(tdd)

encoder = LSTM(56, return_sequences=False, return_state= True)

encoder_outputs, state_h, state_c = encoder(x)
encoder_states = [state_h, state_c]


decoder_inputs = Input(shape=(56, 1 ))
decoder_lstm = LSTM(56, return_sequences=True, return_state=True)
decoder_outputs, h1, c1  = decoder_lstm(decoder_inputs, initial_state = encoder_states)

decoder_lstm = LSTM(56, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_outputs)

decoder_dense = TimeDistributed(Dense(56, activation="hard_sigmoid"))
decoder_outputs = decoder_dense(decoder_outputs)

flat = Flatten()
decoder_outputs = flat(decoder_outputs)

decoder_dense = Dense(1036, activation="hard_sigmoid")
decoder_outputs = decoder_dense(decoder_outputs)

decoder_outputs = tf.reshape(decoder_outputs , (1, 14,74))


# decoder_dense = Dense(14, activation="relu")
# decoder_outputs = decoder_dense(decoder_outputs)


In [ ]:
model = Model([tdd, decoder_inputs], decoder_outputs)

In [ ]:
# model.compile(optimizer='RMSprop', loss="categorical_crossentropy" ,metrics=[tf.keras.metrics.BinaryAccuracy() ,'accuracy'])
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss= losses.CategoricalCrossentropy(from_logits=False)  , metrics=["accuracy" ])
# model.compile(optimizer='RMSprop', loss= losses.SparseCategoricalCrossentropy(from_logits=False ))

In [ ]:
plot_model(model, show_shapes=True)

In [ ]:
np.any(np.isnan(reshaped_input))

In [ ]:
print(reshaped_input.shape, reshaped_output.shape )
history = model.fit([reshaped_input , reshaped_input ] , reshaped_output,  epochs=100, batch_size= 1 , shuffle=True , validation_split=0.2)

In [ ]:
import matplotlib.pyplot as plt  
%matplotlib inline

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig('foo.png')

In [ ]:
int(np.mean(history.history['loss'])) , np.mean(history.history['accuracy']) *100

In [ ]:
np.mean(history.history['loss']) , np.mean(history.history['accuracy'])

In [ ]:
reshaped_input[:1 , : , : ]

In [ ]:
reshaped_output[:1 , :  ]

In [ ]:
model.predict( [reshaped_input[:1 , : , : ] , reshaped_input[:1 , : , : ]]  )

In [ ]:
# https://stackoverflow.com/questions/41908379/keras-plot-training-validation-and-test-set-accuracy
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.show()

In [ ]:
t = model.predict( [reshaped_input[:1 , : , : ] , reshaped_input[:1 , : , : ]]  ) 

In [ ]:
t.min()